## Tarea

En el presente material se intentará explicar de manera lo más clara posible el material dejado en el repositorio del curso

Primera parte: las importaciones

In [ ]:
from typing import Optional
import asyncio
import httpx
from sqlmodel import SQLModel
from pydantic import BaseModel

```from typing import Optional```: Iniciamos con  el tipo ```Optional``` del módulo ```typing```, que se utiliza para indicar que un valor puede ser de un tipo específico o None. Es decir si es entero o un "None"

```import asyncio```: Trae a colación al módulo ```asyncio```, que permite escribir "código concurrente usando la sintaxis ```async/await``` (Fte: Fast Api)". Esta a su vez puede ejecutar tareas de forma asíncrona, como realizar múltiples solicitudes de red simultáneamente. Según FASTAPI, esto permite manejar operaciones lentas sin bloquearse, es decir en otros terminos: simultaneidad = múltiples tareas aparentemente al mismo tiempo y paralelismo = tareas ejecutándose realmente al mismo tiempo

```import httpx```: Esto importa a ```httpx```, esta es una biblioteca para realizar solicitudes HTTP de manera fácil y eficiente (Según la página de la libreria). Es similar a ```requests```, pero con soporte nativo para operaciones asíncronas.

```from sqlmodel import SQLModel```: Importa a la clase ```SQLModel``` del módulo ```sqlmodel```, ya sabemos que "facilita" la creación y manipulación de bases de datos utilizando modelos de datos definidos con clases Python.

```from pydantic import BaseModel```: Esto importa la clase BaseModel de la biblioteca pydantic, nuevamente según la [biblioteca](https://docs.pydantic.dev/latest/why/) que se usa para definir modelos de datos y validación de datos con tipado estático. Se supone que es especialmente útil para asegurarse de que los datos cumplen con ciertas condiciones o formatos.

In [ ]:
# Modelo de Exoplaneta (ahora permite valores nulos en los campos numéricos)
class Exoplanet(BaseModel):
    pl_name: str
    disc_year: int
    pl_orbper: Optional[float]  # Este campo puede ser None
    pl_bmassj: Optional[float]  # Este campo puede ser None

**Clase `Exoplanet`**

La clase `Exoplanet` define un modelo de datos para representar un exoplaneta utilizando **Pydantic** y hereda de `BaseModel`, lo que permite la validación automática de tipos de datos y conversiones.

- **`pl_name: str`**: Atributo de tipo `str` que almacena el nombre del exoplaneta.
- **`disc_year: int`**: Atributo de tipo `int` que representa el año en que se descubrió el exoplaneta.
- **`pl_orbper: Optional[float]`**: Atributo que puede ser un número decimal (`float`) o `None`, indicando el período orbital del exoplaneta.
- **`pl_bmassj: Optional[float]`**: Atributo que puede ser un número decimal (`float`) o `None`, representando la masa del exoplaneta en unidades de masa de Júpiter.

Los campos opcionales (`Optional[float]`) permiten que el valor sea nulo si la página se dispone de la información.


In [ ]:
# Función para obtener datos de la NASA Exoplanet Archive
async def fetch_exoplanets():
    url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync"
    query = """
    SELECT pl_name, disc_year, pl_orbper, pl_bmassj
    FROM ps
    WHERE disc_year >= 2020
    AND ROWNUM <= 5
    """
    params = {
        "query": query,
        "format": "json"
    }

    async with httpx.AsyncClient() as client:
        response = await client.get(url, params=params)
        if response.status_code != 200:
            print(f"Error: {response.status_code}, {response.text}")
            return []

        try:
            data = response.json()
        except ValueError:
            print("La respuesta no es JSON válida. Contenido de la respuesta:")
            print(response.text)
            return []

        return [Exoplanet(**row) for row in data]

La función `fetch_exoplanets` se encarga de obtener datos sobre exoplanetas desde la **NASA Exoplanet Archive** utilizando una consulta SQL y la librería `httpx` de manera asíncrona.

1. **URL y consulta SQL**: Se define la URL de acceso a la API de la NASA Exoplanet Archive y una consulta SQL que selecciona los campos `pl_name`, `disc_year`, `pl_orbper` y `pl_bmassj` de exoplanetas descubiertos a partir de 2020, limitando los resultados a los primeros 5.

2. **Parámetros de la consulta**: Se crea un diccionario `params` con la consulta SQL y el formato de respuesta deseado (JSON).

3. **Solicitud HTTP asíncrona**: Se realiza una solicitud `GET` asíncrona usando `httpx.AsyncClient()` para obtener los datos.

4. **Manejo de errores**: Si la respuesta no tiene un código de estado 200 (OK), se imprime un mensaje de error. Si la respuesta no es JSON válido, también se maneja ese error.

5. **Conversión de datos**: Si la respuesta es válida, se convierte cada fila de datos a un objeto `Exoplanet` usando el constructor `Exoplanet(**row)` y se retorna una lista de estos objetos. Tecnicamente convierte los datos obtenidos en objetos de la clase `Exoplanet`.



In [ ]:
# Función principal que ejecuta la consulta
async def main():
    exoplanets = await fetch_exoplanets()
    for planet in exoplanets:
        print(f"Nombre: {planet.pl_name}, Año de descubrimiento: {planet.disc_year}, "
              f"Periodo orbital (días): {planet.pl_orbper}, Masa (Júpiter): {planet.pl_bmassj}")

if __name__ == "__main__":
    asyncio.run(main())

La función `main` es la función principal que ejecuta la consulta para obtener los exoplanetas y luego muestra sus detalles.

1. **Obtiene los exoplanetas**: Se llama a la función `fetch_exoplanets()` de manera asíncrona utilizando `await`, lo que arroja una lista de objetos `Exoplanet` con los datos de los exoplanetas.

2. **Print**: Se imprime la  información optenida: nombre, año de descubrimiento, período orbital (en días) y masa (en unidades de masa de Júpiter).

3. **Ejecuta el programa**: La parte `if __name__ == "__main__":` verifica que el código solo se ejecute cuando el archivo se corre directamente. Ahí, se ejecuta la función `main()` usando `asyncio.run()`, que maneja la ejecución de tareas asíncronas. (lo saqué en parte en la doc de Fast Api)